In [1]:
import fsspec
import xarray as xr

import plotly.graph_objects as go
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

In [2]:
filepath = 'https://power-analysis-ready-datastore.s3.amazonaws.com/power_901_monthly_meteorology_utc.zarr'
filepath_mapped = fsspec.get_mapper(filepath)

ds = xr.open_zarr(store=filepath_mapped, consolidated=True)
ds

<xarray.Dataset>
Dimensions:          (time: 492, lat: 361, lon: 576)
Coordinates:
  * lat              (lat) float64 -90.0 -89.5 -89.0 -88.5 ... 89.0 89.5 90.0
  * lon              (lon) float64 -180.0 -179.4 -178.8 ... 178.1 178.8 179.4
  * time             (time) datetime64[ns] 1981-01-31 1981-02-28 ... 2021-12-31
Data variables: (12/94)
    CDD0             (time, lat, lon) float64 ...
    CDD10            (time, lat, lon) float64 ...
    CDD18_3          (time, lat, lon) float64 ...
    DISPH            (time, lat, lon) float64 ...
    EVLAND           (time, lat, lon) float64 ...
    EVPTRNS          (time, lat, lon) float64 ...
    ...               ...
    WS50M_MAX_AVG    (time, lat, lon) float64 ...
    WS50M_MIN        (time, lat, lon) float64 ...
    WS50M_MIN_AVG    (time, lat, lon) float64 ...
    WS50M_RANGE      (time, lat, lon) float64 ...
    WS50M_RANGE_AVG  (time, lat, lon) float64 ...
    Z0M              (time, lat, lon) float64 ...
Attributes: (12/37)
    acknowledgement:            The Prediction of Worldwide Energy Resources ...
    comment:                    Version 9.0.1 includes source data resolution.
    conventions:                CF-1.8, ACDD-1.3
    creator_email:              bradley.macpherson@nasa.gov
    creator_name:               Bradley Macpherson
    date_created:               2022-04-11T12:20:00.000000
    ...                         ...
    time_coverage_end:          2021-12-31T00:00:00
    time_coverage_resolution:   P1M
    time_coverage_start:        1981-01-01T00:00:00
    time_standard:              Universal Time Coordinated (UTC)
    title:                      POWER Monthly Meteorology Time Series
    version:                    POWER Version 9.0.1

In [3]:
# SOURCE = /power_901_monthly_meteorology_utc.zarr/
# TSURF -------
# long_name     : Surface Temperature of Land and Snow
# standard_name : Surface_Temperature_of_Land_and_Snow
# units         : K
# valid_max     : 350.0
# valid_min     : 150.0
# valid_range   : 150.0, 350.0]

# GWETTOP -----
# long_name     : Surface Soil Wetness
# standard_name : Surface_Soil_Wetness
# units         : 1
# valid_max     : 1.0
# valid_min     : 0.0
# valid_range   : [0.0, 1.0]

# PRECSNO --------
# long_name     : Snow Precipitation
# standard_name : Snow_Precipitation
# units         : kg m-2 s-1
# valid_max     : 0.0005
# valid_min     : 0.0
# valid_range   : [0.0, 0.0005]

# RH2M -------
# long_name     : Relative Humidity at 2 Meters
# standard_name : Relative_Humidity_at_2_Meters
# units         : %
# valid_max     : 100.0
# valid_min     : 0.0
# valid_range   : [0.0, 100.0]

# WS2M -------
# long_name     : Wind Speed at 2 Meters
# standard_name : Wind_Speed_at_2_Meters
# units         : m/s
# valid_max     : 50.0
# valid_min     : 0.0
# valid_range   : [0.0, 50.0]

# Source = /power_901_daily_precipitation_utc.zarr/
# PRECIPITATIONCAL ------
# long_name : The accumulated precipitation from all available infrared (IR) and microwave (MW) sources.
# units     : mm/day

## Select Columns of Interest

In [4]:
selected = ds[["TSURF", "GWETTOP", "PRECSNO", "RH2M", "WS2M"]]
selected

<xarray.Dataset>
Dimensions:  (time: 492, lat: 361, lon: 576)
Coordinates:
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float64 -180.0 -179.4 -178.8 -178.1 ... 178.1 178.8 179.4
  * time     (time) datetime64[ns] 1981-01-31 1981-02-28 ... 2021-12-31
Data variables:
    TSURF    (time, lat, lon) float64 ...
    GWETTOP  (time, lat, lon) float64 ...
    PRECSNO  (time, lat, lon) float64 ...
    RH2M     (time, lat, lon) float64 ...
    WS2M     (time, lat, lon) float64 ...
Attributes: (12/37)
    acknowledgement:            The Prediction of Worldwide Energy Resources ...
    comment:                    Version 9.0.1 includes source data resolution.
    conventions:                CF-1.8, ACDD-1.3
    creator_email:              bradley.macpherson@nasa.gov
    creator_name:               Bradley Macpherson
    date_created:               2022-04-11T12:20:00.000000
    ...                         ...
    time_coverage_end:          2021-12-31T00:00:00
    time_coverage_resolution:   P1M
    time_coverage_start:        1981-01-01T00:00:00
    time_standard:              Universal Time Coordinated (UTC)
    title:                      POWER Monthly Meteorology Time Series
    version:                    POWER Version 9.0.1

## Select Data from 2015-2021
We also drop NA values which are readings of the sea.

In [5]:
all_data = selected.sel(time=slice('20-01-01', '2022-12-31')).to_dataframe().reset_index()
all_data

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M
0,2001-01-31,-90.0,-180.000,NaN,1.0,0.000000,94.2500,2.203125
1,2001-01-31,-90.0,-179.375,NaN,1.0,0.000000,94.2500,2.203125
2,2001-01-31,-90.0,-178.750,NaN,1.0,0.000000,94.2500,2.210938
3,2001-01-31,-90.0,-178.125,NaN,1.0,0.000000,94.2500,2.218750
4,2001-01-31,-90.0,-177.500,NaN,1.0,0.000000,94.2500,2.226562
...,...,...,...,...,...,...,...,...
52399867,2021-12-31,90.0,176.875,NaN,1.0,0.000005,89.9375,4.117188
52399868,2021-12-31,90.0,177.500,NaN,1.0,0.000005,89.9375,4.125000
52399869,2021-12-31,90.0,178.125,NaN,1.0,0.000005,89.9375,4.132812
52399870,2021-12-31,90.0,178.750,NaN,1.0,0.000005,89.9375,4.140625


In [6]:
all_data.describe()

,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M
count,5.239987e+07,5.239987e+07,1.483675e+07,4.877107e+07,5.239987e+07,5.239987e+07,5.239987e+07
mean,0.000000e+00,-3.125000e-01,2.829015e+02,8.912040e-01,4.444090e-06,8.175647e+01,5.176610e+00
std,5.210566e+01,1.039229e+02,1.808677e+01,2.251209e-01,9.250743e-06,1.447742e+01,2.340508e+00
min,-9.000000e+01,-1.800000e+02,2.208750e+02,7.812500e-03,0.000000e+00,6.125000e+00,0.000000e+00
25%,-4.500000e+01,-9.015625e+01,2.713906e+02,9.140625e-01,0.000000e+00,7.831250e+01,3.585938e+00
50%,-1.797510e-13,-3.125000e-01,2.864219e+02,1.000000e+00,0.000000e+00,8.331250e+01,5.210938e+00
75%,4.500000e+01,8.953125e+01,2.981172e+02,1.000000e+00,7.629395e-06,9.081250e+01,6.718750e+00
max,9.000000e+01,1.793750e+02,3.174609e+02,1.000000e+00,5.798340e-04,1.000000e+02,1.837500e+01


In [7]:
all_data.dropna(inplace=True)
all_data.reset_index(inplace=True)
all_data

,index,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M
0,34806,2001-01-31,-60.0,-26.250,274.882812,1.000000,0.000023,89.3750,6.140625
1,35378,2001-01-31,-59.5,-28.750,274.898438,1.000000,0.000023,90.4375,5.953125
2,35379,2001-01-31,-59.5,-28.125,274.898438,0.953125,0.000023,90.1250,5.929688
3,35380,2001-01-31,-59.5,-27.500,274.890625,0.914062,0.000023,89.7500,5.968750
4,35381,2001-01-31,-59.5,-26.875,274.882812,0.890625,0.000023,89.3125,6.039062
...,...,...,...,...,...,...,...,...,...
13799767,52392612,2021-12-31,84.0,-37.500,244.812500,0.984375,0.000005,97.1250,5.078125
13799768,52392613,2021-12-31,84.0,-36.875,244.812500,0.984375,0.000005,97.3750,5.117188
13799769,52392614,2021-12-31,84.0,-36.250,244.812500,0.984375,0.000006,97.6250,5.156250
13799770,52392615,2021-12-31,84.0,-35.625,244.812500,0.992188,0.000006,97.7500,5.195312


In [8]:
all_data.drop(columns=["index"], inplace=True)

## Convert Kelvin to Fahrenheit

In [9]:
all_data["F_TSURF"] = (9/5) * (all_data["TSURF"] - 273) + 32

all_data

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M,F_TSURF
0,2001-01-31,-60.0,-26.250,274.882812,1.000000,0.000023,89.3750,6.140625,35.389063
1,2001-01-31,-59.5,-28.750,274.898438,1.000000,0.000023,90.4375,5.953125,35.417187
2,2001-01-31,-59.5,-28.125,274.898438,0.953125,0.000023,90.1250,5.929688,35.417187
3,2001-01-31,-59.5,-27.500,274.890625,0.914062,0.000023,89.7500,5.968750,35.403125
4,2001-01-31,-59.5,-26.875,274.882812,0.890625,0.000023,89.3125,6.039062,35.389063
...,...,...,...,...,...,...,...,...,...
13799767,2021-12-31,84.0,-37.500,244.812500,0.984375,0.000005,97.1250,5.078125,-18.737500
13799768,2021-12-31,84.0,-36.875,244.812500,0.984375,0.000005,97.3750,5.117188,-18.737500
13799769,2021-12-31,84.0,-36.250,244.812500,0.984375,0.000006,97.6250,5.156250,-18.737500
13799770,2021-12-31,84.0,-35.625,244.812500,0.992188,0.000006,97.7500,5.195312,-18.737500


In [10]:
all_data['fips_code'] = None
all_data['county_name'] = None

In [11]:
all_data

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M,F_TSURF,fips_code,county_name
0,2001-01-31,-60.0,-26.250,274.882812,1.000000,0.000023,89.3750,6.140625,35.389063,None,None
1,2001-01-31,-59.5,-28.750,274.898438,1.000000,0.000023,90.4375,5.953125,35.417187,None,None
2,2001-01-31,-59.5,-28.125,274.898438,0.953125,0.000023,90.1250,5.929688,35.417187,None,None
3,2001-01-31,-59.5,-27.500,274.890625,0.914062,0.000023,89.7500,5.968750,35.403125,None,None
4,2001-01-31,-59.5,-26.875,274.882812,0.890625,0.000023,89.3125,6.039062,35.389063,None,None
...,...,...,...,...,...,...,...,...,...,...,...
13799767,2021-12-31,84.0,-37.500,244.812500,0.984375,0.000005,97.1250,5.078125,-18.737500,None,None
13799768,2021-12-31,84.0,-36.875,244.812500,0.984375,0.000005,97.3750,5.117188,-18.737500,None,None
13799769,2021-12-31,84.0,-36.250,244.812500,0.984375,0.000006,97.6250,5.156250,-18.737500,None,None
13799770,2021-12-31,84.0,-35.625,244.812500,0.992188,0.000006,97.7500,5.195312,-18.737500,None,None


## Get US Counties Coordinates

In [12]:
us_counties = pd.read_csv("./us_counties.csv")
us_counties

,fips_code,name,lng,lat
0,1059,Franklin,-87.843283,34.442381
1,13111,Fannin,-84.319296,34.864126
2,19109,Kossuth,-94.206898,43.204140
3,40115,Ottawa,-94.810589,36.835878
4,42115,Susquehanna,-75.800905,41.821277
...,...,...,...,...
3228,12029,Dixie,-83.158705,29.608068
3229,18017,Cass,-86.346207,40.761660
3230,26091,Lenawee,-84.066412,41.894694
3231,72003,Aguada,-67.175247,18.360392


## Map Coordinates to Counties

In [14]:
def get_two_closest_readings(lat, lon, threshold=0.5):
    # Get all entries where the difference in longitude is less than or equal to the threshold
    closest_longitudes = all_data[(all_data['lon'] - lon).abs() <= threshold]
    # Get all entries where the difference in longitude is less than or equal to the threshold
    return closest_longitudes[(closest_longitudes['lat'] - lat).abs() <= threshold].index
    
# Assign counties
for fips, name, lon, lat in tqdm(us_counties.values):
    indices = get_two_closest_readings(lat, lon)
    if len(indices) > 0:
        all_data.loc[indices, "fips_code"] = fips
        all_data.loc[indices, "county_name"] = name

100%|██████████| 3233/3233 [02:15<00:00, 23.81it/s]


In [15]:
# Remove readings without an assigned county
all_data.dropna(inplace=True)

In [17]:
all_data

,time,lat,lon,TSURF,GWETTOP,PRECSNO,RH2M,WS2M,F_TSURF,fips_code,county_name
7140,2001-01-31,-14.5,-170.625,302.789062,0.890625,0.000000,80.6875,4.070312,85.620313,60010,Eastern
7141,2001-01-31,-14.5,-170.000,302.421875,0.968750,0.000000,81.0625,4.164062,84.959375,60020,Manu'a
7142,2001-01-31,-14.5,-169.375,302.421875,0.968750,0.000000,81.1250,4.265625,84.959375,60020,Manu'a
7293,2001-01-31,-14.0,-171.250,302.195312,0.867188,0.000000,81.0625,4.000000,84.551563,60050,Western
7294,2001-01-31,-14.0,-170.625,302.789062,0.882812,0.000000,80.7500,4.125000,85.620313,60010,Eastern
...,...,...,...,...,...,...,...,...,...,...,...
13791706,2021-12-31,67.5,-159.375,247.812500,0.945312,0.000039,96.0000,4.429688,-13.337500,2188,Northwest Arctic
13793097,2021-12-31,69.0,-153.750,245.625000,0.882812,0.000013,91.9375,3.085938,-17.275000,2185,North Slope
13793098,2021-12-31,69.0,-153.125,245.515625,0.867188,0.000012,92.1250,3.101562,-17.471875,2185,North Slope
13793535,2021-12-31,69.5,-153.750,245.226562,0.796875,0.000012,95.6250,2.859375,-17.992188,2185,North Slope


In [16]:
all_data.to_csv("us_data.csv")